# Importing libraries

In [9]:
# from google.colab import files
import os
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np
from PIL import Image as Img
import cv2
from skimage.transform import resize
import torch

ModuleNotFoundError: No module named 'tensorflow'

In [3]:
!pip3 install torch torchvision

In [4]:
# print('making images directory')
# !mkdir images
# print('making results directory')
# !mkdir results

In [5]:
# import image here

In [6]:
!python -W ignore u2net_test.py

Traceback (most recent call last):
  File "u2net_test.py", line 3, in <module>
    import torch
ModuleNotFoundError: No module named 'torch'


In [7]:
# get the names of the images that were uploaded, removing .jpg
image_dir = os.path.join(os.getcwd(), 'images')
names = [name[:-4] for name in os.listdir(image_dir)]
names

['.DS_S', 'backgro', '2337127678_001']

In [8]:
# this notebook only uses the first of the uploaded images
name = names[0]
name

'.DS_S'

In [ ]:
# display result that U-2-Net created
output = load_img('/content/U-2-Net/results/'+name+'.png')
display(output)

# Background Removal

In [ ]:
# convert output to numpy array and rescale(255 for RBG images)
RESCALE = 255
out_img = img_to_array(output)
out_img /= RESCALE
import matplotlib.pyplot as plt
plt.imshow(out_img)

In [3]:
# define the cutoff threshold below which, background will be removed.
THRESHOLD = 0.9

# refine the output
out_img[out_img > THRESHOLD] = 1
out_img[out_img <= THRESHOLD] = 0
plt.imshow(out_img)

NameError: ignored

In [ ]:
# convert the rbg image to an rgba image and set the zero values to transparent
shape = out_img.shape
a_layer_init = np.ones(shape = (shape[0],shape[1],1))
mul_layer = np.expand_dims(out_img[:,:,0],axis=2)
a_layer = mul_layer*a_layer_init
rgba_out = np.append(out_img,a_layer,axis=2)
display(Img.fromarray((rgba_out*RESCALE).astype('uint8'), 'RGBA'))

In [ ]:
# load and convert input to numpy array and rescale(255 for RBG images)
input = load_img('/content/U-2-Net/images/'+name+'.png')
inp_img = img_to_array(input)
inp_img /= RESCALE
plt.imshow(inp_img)

In [ ]:
# since the output image is rgba, convert this also to rgba, but with no transparency
a_layer = np.ones(shape = (shape[0],shape[1],1))
rgba_inp = np.append(inp_img,a_layer,axis=2)
plt.imshow(rgba_inp)

In [ ]:
# simply multiply the 2 rgba images to remove the backgound
rem_back = (rgba_inp*rgba_out)
rem_back_scaled = Img.fromarray((rem_back*RESCALE).astype('uint8'), 'RGBA')
display(rem_back_scaled)
# --same, but using cv2--
#rem_back_cv = rem_back[:,:,[2,1,0,3]]
#from google.colab.patches import cv2_imshow
#cv2_imshow(rem_back_cv*RESCALE)

In [ ]:
# save the resulting image to colab
rem_back_scaled.save(name+'_background_removed.png')

In [ ]:
# download the result
files.download('/content/U-2-Net/'+name+'_background_removed.png')

# Change Background

In [ ]:
# get the names of the images that were uploaded, removing .jpg
image_dir = os.path.join(os.getcwd(), 'images/backgrounds')
backgrounds = [background[:-4] for background in os.listdir(image_dir)]
backgrounds

In [ ]:
# this notebook only uses the first of the uploaded images
background = backgrounds[0] # `black_bg.png`
background

In [ ]:
# load and convert background to numpy array and rescale(255 for RBG images)
background_input = load_img('/content/U-2-Net/images/backgrounds/'+background+'.png')
background_inp_img = img_to_array(background_input)
background_inp_img /= RESCALE
plt.imshow(background_inp_img)

In [ ]:
# get dimensions of background (original image will be resized to dimensions of background image in this notebook)
background_height = background_inp_img.shape[0]
background_width = background_inp_img.shape[1]
background_height,background_width

In [ ]:
# resize the image
resized_rem_back  = resize(rem_back, (background_height,background_width))
plt.imshow(resized_rem_back)

In [ ]:
# create a new array which will store the final result
output_chbg = np.zeros((background_height, background_width, 3))
output_chbg.shape

In [ ]:
# using the following o[c] = b[c]*(1-i[t])+i[c] {where o - output image, c - channels from 1-3, i - input image with background removed, t - transparent channel}, obtain values for the final result
output_chbg[:,:,0] = background_inp_img[:,:,0]*(1-resized_rem_back[:,:,3])+resized_rem_back[:,:,0]
output_chbg[:,:,1] = background_inp_img[:,:,1]*(1-resized_rem_back[:,:,3])+resized_rem_back[:,:,1]
output_chbg[:,:,2] = background_inp_img[:,:,2]*(1-resized_rem_back[:,:,3])+resized_rem_back[:,:,2]
plt.imshow(output_chbg)

In [ ]:
# rescale
output_chbg_scaled = Img.fromarray((output_chbg*RESCALE).astype('uint8'), 'RGB')
display(output_chbg_scaled)

In [ ]:
# save the resulting image to colab
output_chbg_scaled.save(name+'_background_changed.png')